| Model      | AP50 normal | AP50 mitoses | P50 normal | R50 normal | P50 mitoses | R50 mitoses |   AP50    |   AP75    |   AP50:95   |
|----------- | ----------- | ------------ | ---------- | ---------- | ----------- | ----------- |   -----   |   -----   |   -------   |
| RetinaNet  | **75.25**   |   56.47      |   81.87    | **75.87**  |   54.33     |   65.71     |   65.86   | **19.17** |   29.56     |
| R3Det      |   74.97     |   63.21      | **83.93**  |   73.61    |   66.66     |   64.76     | **69.09** |   17.91   | **30.89**   |
| DCL        |   71.81     | **63.35**    |   77.89    |   72.71    | **66.98**   | **67.62**   |   67.58   |   17.74   |   28.48     |
| CSL        |   72.42     |   53.98      |   75.37    |   73.50    |   51.93     |   63.81     |   63.20   |   18.59   |   27.00     |
| Ours       |   59.41     |   35.58      |   68.02    |   67.64    |   52.50     |   40.00     |   47.50   |    5.31   |   16.37     |

### Compilations

In [ ]:
!apt update && apt install -y libsm6 libxext6
!apt-get install -y libxrender-dev

In [ ]:
%cd /workdir/msc/RotationDetection/libs/utils/cython_utils
!rm *.so
!rm *.c
!rm *.cpp
!python setup.py build_ext --inplace

In [ ]:
%cd /workdir/msc/RotationDetection/libs/utils/
!rm *.so
!rm *.c
!rm *.cpp
!python setup.py build_ext --inplace

### Dataset

In [ ]:
# change train and test paths inside the script
%cd /workdir/msc/RotationDetection/dataloader/dataset/UFRGS_CELL
!python convert_ann2dota.py

In [ ]:
# change root path inside the script
%cd /workdir/msc/RotationDetection/dataloader/dataset/UFRGS_CELL
!python convert2jpg.py

In [ ]:
# change train path inside the script
%cd /workdir/msc/RotationDetection/dataloader/dataset/UFRGS_CELL
!python data_crop.py

In [ ]:
# change test path inside the script
%cd /workdir/msc/RotationDetection/dataloader/dataset/UFRGS_CELL
!python txt2xml.py

In [ ]:
%cd /workdir/msc/RotationDetection/dataloader/dataset/  
!python convert_data_to_tfrecord.py --VOC_dir='/workdir/datasets/msc/UFRGS_CELL_2classes/crop' \
                                    --xml_dir='labeltxt' \
                                    --image_dir='images' \
                                    --save_name='train'  \
                                    --img_format='.png'  \
                                    --save_dir='/workdir/datasets/msc/' \
                                    --dataset='UFRGS_CELL' \

## Train

In [ ]:
# change the path to the detector!
%cd /workdir/msc/RotationDetection/tools/csl
!python train.py

************************************************************************
2021-07-05 01:13:00: global_step:126140  current_step:126140
speed: 0.306s, remaining training time: 00:06:16:33
total_losses:1.064
cls_loss:0.045
reg_loss:0.234
angle_cls_loss:0.784

************************************************************************
2021-07-05 01:13:06: global_step:126160  current_step:126160
speed: 0.288s, remaining training time: 00:05:54:11
total_losses:1.295
cls_loss:0.070
reg_loss:0.229
angle_cls_loss:0.997

************************************************************************
2021-07-05 01:13:13: global_step:126180  current_step:126180
speed: 0.289s, remaining training time: 00:05:55:15
total_losses:1.039
cls_loss:0.037
reg_loss:0.327
angle_cls_loss:0.675

************************************************************************
2021-07-05 01:13:25: global_step:126220  current_step:126220
speed: 0.292s, remaining training time: 00:05:58:34
total_losses:0.732
cls_loss:0.042
reg_loss:

## Test

In [2]:
%cd /workdir/msc/RotationDetection/tools/dcl
!python test_ufrgscell.py --img_dir='/workdir/datasets/msc/UFRGS_CELL_2classes/test/imgs'  \
                          --gpu=0 \
                          --image_ext='.jpg' \
                          --test_annotation_path='/workdir/datasets/msc/UFRGS_CELL_2classes/test/xml_rotdet'

/workdir/msc/RotationDetection/tools/dcl
/workdir/msc/RotationDetection


******************************
 CHECKPOINT PATH: /workdir/msc/RotationDetection/output/trained_weights/DLCv3_UFRGS_CELL_smooth_l1_loss/UFRGS_CELL_199999model.ckpt

******************************
model restore from : /workdir/msc/RotationDetection/output/trained_weights/DLCv3_UFRGS_CELL_smooth_l1_loss/UFRGS_CELL_199999model.ckpt

**********
Restored model
/workdir/msc/RotationDetection/output/trained_weights/DLCv3_UFRGS_CELL_smooth_l1_loss/UFRGS_CELL_199999model.ckpt
**********
Eval image exp_M3_Mauricio_mcf7_frame_17: 100%|█| 30/30 [00:15<00:00,  1.99it/s]
********************
rotation eval:
Writing normal_cell VOC resutls file
Writing mitoses VOC resutls file
Threshold:  0.5
cls : normal_cell|| Recall: 0.8162344973886871 || Precison: 0.3958447238928376|| AP: 0.7181695206633543
F1:0.7521815944334501 P:0.7789855072463768 R:0.7271702359332918
cls : mitoses|| Recall: 0.809523801814059 || Precison: 0.1650485436893204

## Inference

In [ ]:
import os
import cv2
from glob import glob
import numpy as np
import matplotlib.pyplot as plt

path_labels = '/workdir/datasets/msc/UFRGS_CELL_2classes/test/annotations/dota_format'
path_draws = '/workdir/msc/RotationDetection/tools/gwd/test_ufrgscell/RetinaNet_UFRGS_CELL_smooth_l1_loss/ufrgscell_img_vis'

path_labels = sorted(glob(os.path.join(path_labels, '*.txt')))
path_draws  = sorted(glob(os.path.join(path_draws, '*.jpg')))

for i in range(len(path_labels)):
    path_ann = path_labels[i]
    path_img = path_draws[i]

    img = cv2.imread(path_img)[:,:,::-1]
    h,w,c = img.shape

    labels = []
    for lb in open(path_ann, 'r').read().split('\n')[2:-1]:
        lb = np.reshape(list(map(float, lb.split(' ')[:-2])), (-1,2))
        labels.append(lb)
    labels = np.int0(labels)

    draw = np.copy(img)
    cv2.drawContours(draw, labels, -1, (255,0,0), 2)

    plt.figure(figsize=(15,15))
    plt.imshow(draw)